In [1]:
import black
import jupyter_black

jupyter_black.load(
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

DEBUG:jupyter_black:config: {'line_length': 79, 'target_versions': {<TargetVersion.PY310: 10>}}


In [2]:
import random
import hashlib as h
import time
import numpy as np

hash_data_type = np.dtype("S4")
num_bytes = 4
zeros = 0
list_of_password_extractions = []
list_of_colliding_hashes = []

In [3]:
"""Write the generated results to a .txt file"""


def writeResults(p):
    f = open("RainbowDataTemp.txt", "a")
    for i in range(0, 12):
        f.write(str(p[i]) + "--")
    f.write("\n")
    f.close()

In [4]:
"""h(x) - apply only hash function"""


def apply_hash(current):
    current = h.sha1(current).digest()[:num_bytes]
    return current

In [5]:
"""f_i(x) = g_i(h(x)) - apply hash function then reduction function (XOR with i)"""


def apply_full_function(i, current):
    current = apply_hash(current)
    current = bytes(x ^ y for x, y in zip(i, current))[
        :num_bytes
    ]  # we always workin with the reduced val?
    return current

In [6]:
"""Generate random password of a given size"""


def get_random_password(size):
    new = random.getrandbits(size)
    password = new.to_bytes(num_bytes, "little")
    hashed_value = apply_full_function(zeros, password)
    return (password, hashed_value)

In [7]:
"""Generate a list of random hashes to attempt to invert"""


def generate_passwords(num, num_bits):
    my_passwords = list()
    for i in range(num):
        my_passwords.append(get_random_password(num_bits))
    return my_passwords

In [8]:
"""Generate a single random start point, returns the start point alongside empty values for end point and chain
length in the form of a 3 element triple to be stored in the Hellman Table"""


def get_start_point(input_size):
    new = random.getrandbits(input_size)
    sp = new.to_bytes(num_bytes, "little")
    return sp

In [9]:
"""Repeatedly call the get_start_point function in order to generate all requiered start points"""


def gen_start_points(num_chains, input_size):
    global num_bytes
    num_bytes = input_size // 8
    start_points = [0 for i in range(num_chains)]
    start_points = np.array(start_points, dtype=hash_data_type)

    for i in range(0, num_chains):
        start_points[i] = get_start_point(input_size)

    return start_points

In [10]:
"""Calculate the end point for a given start point"""


def get_end_point(start_point, chain_length):
    current = start_point
    for i in range(0, chain_length):
        current = apply_full_function(i.to_bytes(num_bytes, "little"), current)
    return current

In [11]:
"""Remove duplicate end points and regenerate the chains for the shorter duplicates"""


def find_duplicates(end_points):
    previous = end_points[0]
    duplicate_indices = list()
    for i in range(0, len(end_points) - 1):
        if previous == end_points[i + 1]:
            duplicate_indices.append(i + 1)
        previous = end_points[i]
    return duplicate_indices

In [12]:
"""Generate the specified number of end points and also removes duplicate rows."""


def gen_end_points(start_points, chain_length, input_size):
    global num_bytes
    duplicates_exist = True
    num_chains = len(start_points)

    end_points = [0 for i in range(num_chains)]
    end_points = np.array(end_points, dtype=hash_data_type)

    for i in range(0, num_chains):
        end_points[i] = get_end_point(start_points[i], chain_length)

    while duplicates_exist:
        order = np.argsort(end_points)

        # sort start and end points
        start_points = start_points[order]
        end_points = end_points[order]
        replace = find_duplicates(end_points)

        if len(replace) != 0:
            for chain in replace:
                start_points[chain] = get_start_point(input_size)
                end_points[chain] = get_end_point(
                    start_points[chain], chain_length
                )
        else:
            duplicates_exist = False

    return start_points, end_points

In [13]:
"""Method to recalculate chain up to a specified stop point"""


def find_chain_entry(chain_number, stop_point, start_points):
    current = start_points[chain_number]
    for i in range(stop_point):
        current = apply_full_function(i.to_bytes(num_bytes, "little"), current)
    return current

In [14]:
"""Method to search through the chains of the rainbow table, returns true or false and
the number of collisions / false alarms that occurred"""


def search_chains(tp, y, start_points, end_points, chain_length):

    true_password = tp
    num_chains = len(start_points)
    false_alarms = 0
    gen_hashes = 0
    false_alarm_hashes = 0
    success_hashes = 0

    global list_of_password_extractions
    global list_of_colliding_hashes

    hash_of_password = y

    for i in range(chain_length, 0, -1):

        hash_of_password = y

        hash_of_password = bytes(
            x ^ y
            for x, y in zip(
                (i - 1).to_bytes(num_bytes, "little"), hash_of_password
            )
        )[:num_bytes]

        for j in range(i, chain_length):
            hash_of_password = apply_full_function(
                (j).to_bytes(num_bytes, "little"), hash_of_password
            )

        gen_hashes += chain_length - i

        for x in range(0, num_chains):
            current_end_point = end_points[x]
            if current_end_point == hash_of_password:
                password_guess = find_chain_entry(
                    chain_number=x, stop_point=i - 1, start_points=start_points
                )
                temp_hash_count = i - 1

                if password_guess == true_password:
                    success_hashes = temp_hash_count
                    list_of_password_extractions.append(
                        [
                            tp,
                            y,
                            password_guess,
                        ]
                    )
                    return (
                        True,
                        false_alarms,
                        gen_hashes,
                        false_alarm_hashes,
                        success_hashes,
                    )

                if apply_hash(password_guess) == apply_hash(true_password):
                    success_hashes = temp_hash_count
                    list_of_colliding_hashes.append(
                        [
                            tp,
                            y,
                            password_guess,
                        ]
                    )
                    return (
                        True,
                        false_alarms,
                        gen_hashes,
                        false_alarm_hashes,
                        success_hashes,
                    )

                else:
                    false_alarm_hashes += temp_hash_count
                    false_alarms += 1

    return (
        False,
        false_alarms,
        gen_hashes,
        false_alarm_hashes,
        success_hashes,
    )

In [15]:
"""Rainbow table method"""


def rainbow_table(no_chains, chain_len, no_iterations, hash_size):

    false_alarms = 0
    inverse_success = 0
    gen_hashes = 0
    false_alarm_hashes = 0
    success_hashes = 0
    global zeros
    global num_bytes
    global hash_data_type

    num_bytes = hash_size // 8
    zeros = 0
    zeros = zeros.to_bytes(num_bytes, "little")
    data_type_string = "S" + str(num_bytes)
    hash_data_type = np.dtype(data_type_string)

    # time Hellman table creation
    start = time.time()
    my_start_points = gen_start_points(no_chains, hash_size)
    my_start_points, my_end_points = gen_end_points(
        my_start_points, chain_len, hash_size
    )
    end = time.time()
    table_time = end - start

    my_passwords = generate_passwords(no_iterations, hash_size)

    # time search algorithm
    start = time.time()
    for i in range(no_iterations):
        x = search_chains(
            my_passwords[i][0],
            my_passwords[i][1],
            my_start_points,
            my_end_points,
            chain_len,
        )
        false_alarms += x[1]
        if x[0]:
            inverse_success += 1
        gen_hashes += x[2]
        false_alarm_hashes += x[3]
        success_hashes += x[4]
    end = time.time()
    search_time = end - start

    accuracy = inverse_success / no_iterations
    return (
        accuracy,
        false_alarms,
        table_time,
        search_time,
        gen_hashes,
        false_alarm_hashes,
        success_hashes,
    )

In [16]:
"""Master method which calls rainbow table and write results as well as times the
length of execution"""


def masterMethod(p):
    parameters = p
    start = time.time()
    my_results = rainbow_table(
        no_chains=parameters[0],
        chain_len=parameters[1],
        no_iterations=parameters[2],
        hash_size=parameters[3],
    )
    end = time.time()

    # accuracy
    parameters.append(my_results[0] * 100)

    # false_alarms
    parameters.append(my_results[1])

    # general hashes
    parameters.append(my_results[4])

    # false alarm hashes
    parameters.append(my_results[5])

    # success hashes
    parameters.append(my_results[6])

    # table generation time
    parameters.append(my_results[2])

    # search algorithm time
    parameters.append(my_results[3])

    # total execution time
    total_time = end - start
    parameters.append(total_time)

    # write file
    writeResults(parameters)


parameters = [1681, 41, 100, 16]
masterMethod(parameters)

In [17]:
list_of_password_extractions

[[b'\xde\xdc', b'x#', b'\xde\xdc'],
 [b'ke', b'\x15\xd5', b'ke'],
 [b'\x03<', b'\xf5\xd0', b'\x03<'],
 [b'B\x86', b'\x81\xe8', b'B\x86'],
 [b'\xa4\x9c', b'w\xa4', b'\xa4\x9c'],
 [b'\x90\x8e', b'\xde\x98', b'\x90\x8e'],
 [b',G', b'\x91\x80', b',G'],
 [b'R\x0e', b'$\xfc', b'R\x0e'],
 [b'\x8f\xdb', b'9\xba', b'\x8f\xdb'],
 [b'\x04\x11', b'"%', b'\x04\x11'],
 [b'\xdaL', b'\n\xa8', b'\xdaL'],
 [b'f\x1a', b'\xfb\x8c', b'f\x1a'],
 [b'\x8e\xf2', b'Nx', b'\x8e\xf2'],
 [b'HR', b'\xf1\x87', b'HR'],
 [b'\x85H', b'\x03\xb7', b'\x85H'],
 [b't\xdc', b'm\x0f', b't\xdc'],
 [b':\x9c', b'\xd9\x91', b':\x9c'],
 [b'H\x17', b'\xc5\x98', b'H\x17'],
 [b'\xff\xe7', b'\xed\xf6', b'\xff\xe7'],
 [b'\xc1\xeb', b'\xf0s', b'\xc1\xeb'],
 [b'\xa3\xdf', b'\xfa\xb3', b'\xa3\xdf'],
 [b'wq', b'\xf8\x8b', b'wq'],
 [b'vZ', b'\xeb<', b'vZ'],
 [b'\x1e]', b'\xf3w', b'\x1e]'],
 [b'^\x88', b'lR', b'^\x88'],
 [b'Ly', b'\x08N', b'Ly'],
 [b'\x07\xea', b'|\xd5', b'\x07\xea'],
 [b'(\xbd', b'"\x0c', b'(\xbd'],
 [b'\x97\x80', b'&\n', b

In [18]:
list_of_colliding_hashes

[[b'\xb5_', b'3\xb1', b'\x00\\'],
 [b'nf', b'\x0c\xdd', b's\xcf'],
 [b'\xdbr', b'p\x18', b'\xd7\x1b'],
 [b'\xc3&', b'\x1a)', b'\x0e3'],
 [b'\xcf/', b'\xc2\xa4', b'\xded'],
 [b'\xa6\x8c', b'\xd4\x16', b'\xb5|'],
 [b'\xe7\xf9', b'\xf8G', b'\x07P'],
 [b'\xb8r', b'\xd4\x16', b'\xb5|'],
 [b'=\x87', b'\x0c\xcf', b'\xce\xe7'],
 [b'\x1b\x93', b'\x16\xcd', b'\xfbs'],
 [b'F\xec', b'#!', b'\xb5\xf2'],
 [b'F\x82', b'\x80\xc3', b'\x88"'],
 [b'\xc8l', b'W\xd8', b'\xc7\x81'],
 [b'\xaa%', b'A\xc8', b'\x00\xaa'],
 [b'\xcf\xb6', b'm;', b'\xd1\xb5'],
 [b'\x80"', b',\xb4', b'_\xc8'],
 [b'X\xb0', b'\x06\x91', b'\xa0Q'],
 [b'\xd6\xf6', b'L\x93', b'\xa8p'],
 [b'`\xb1', b'\x90q', b'\x9eP'],
 [b'\xe9\x8d', b'\x94\xce', b'\x05\x98'],
 [b'b\xd9', b'\x9fY', b'\x83\xbc'],
 [b'\x93\xe2', b'>\x97', b'\x86\x81'],
 [b']\xc4', b'\x199', b'\x99\x7f'],
 [b'\xc3q', b'\x19q', b'\x89\xd8'],
 [b'A]', b'0\x12', b'\x87\xda']]